In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
%matplotlib inline

In [6]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras

## 데이터 불러오기

In [8]:
Fold       = 5
Fs         = 12800  # Sampling Frequency
NoOfSensor = 1
# 이미지 데이터를 분석하는 경우 한 픽셀 당 rgb 3개 값이 있으니 NoOfSensor = 3이 되어야 한다.

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/CNN_Training_Fold%d'%(i+1)
    path2 = './K_FoldData/CNN_Validation_Fold%d'%(i+1)
    c1 = 'temp_Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'temp_Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
temp_Training_All  = np.array(pd.read_csv('./K_FoldData/CNN_Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(temp_Training_Fold1.shape)
print(temp_Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(temp_Training_All.shape)

(256, 2774)
(64, 2774)
(256, 2)
(64, 2)
(320, 2774)


## STFT 실행 (Spectrogram 생성)

In [9]:
# STFT test

f,t,Sxx = signal.spectrogram(temp_Training_All, Fs, nperseg=78)
print("STFT Result            :", Sxx.shape)

# Data Reshape for CNN
Training_All = Sxx.reshape(Sxx.shape[0], Sxx.shape[1], Sxx.shape[2], NoOfSensor)
print("Reshape_for_CNN Result :", Training_All.shape)

STFT Result            : (320, 40, 40)
Reshape_for_CNN Result : (320, 40, 40, 1)


In [10]:
for i in range(Fold):
    s1 = 'f,t,Sxx1 = signal.spectrogram(temp_Training_Fold%d, Fs, nperseg=78)'   %(i+1)
    s2 = 'f,t,Sxx2 = signal.spectrogram(temp_Validation_Fold%d, Fs, nperseg=78)' %(i+1)
    exec(s1)
    exec(s2)
    
    s3 = 'Training_Fold%d   = Sxx1.reshape(Sxx1.shape[0], Sxx1.shape[1], Sxx1.shape[2], NoOfSensor)' %(i+1)
    s4 = 'Validation_Fold%d = Sxx2.reshape(Sxx2.shape[0], Sxx2.shape[1], Sxx2.shape[2], NoOfSensor)' %(i+1)
    exec(s3)
    exec(s4)


print(f.shape, t.shape, Training_Fold1.shape, Validation_Fold1.shape)

(40,) (40,) (256, 40, 40, 1) (64, 40, 40, 1)


## CNN(Convolution Neural Network) hyperparameter 설정

In [11]:
learningRate  = 0.001
iteration     = 500

## Keras 기반 CNN 구조(Architecture) 설계

In [12]:
def CNN_model(input_data):
    #모델을 정의함
    model = keras.Sequential() 
    # 레이어를 추가함 (convolution layer)
    '''
    4: 첫 레이어의 퍼셉트론 개수 -> bias 또한 4개
    kernel_size: 차원 축소하는 필터 행렬 크기 (3,3) -> 3*3*4 = 36개의 weight가 생기는 것
    -> 36+4 = 40개의 hyperparameter (최적화해야 할 대상)이 생기는 것
    strides: 몇 칸씩 넘어가면서 줄일거냐
    padding = 'same': 행렬 주변에 0으로 채워서 가장자리 값도 계산 많이 하게 하는 것
    activation : 활성함수 뭐로 쓸거냐
    
    
    '''
    model.add(keras.layers.Conv2D(4, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu',
                                  input_shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3]))) # Convolution layer 1
    #pooling: 차원 축소하기 원래는 convolution
    model.add(keras.layers.MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))                                  # Pooling layer 1
    model.add(keras.layers.Conv2D(8, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'))   # Convolution layer 2
    
    # GlobalAveragePooling: ANN을 대체할 수 있는 함수
    model.add(keras.layers.GlobalAveragePooling2D())                                                          # Global Average Pooling
    # Output Layer
    model.add(keras.layers.Dense(units = 2, activation='softmax'))                                            

    # ANN에서 썼던 것과 똑같음
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

In [14]:
CNN_model(Training_Fold1).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 40, 4)         40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 8)         296       
_________________________________________________________________
global_average_pooling2d (Gl (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 18        
Total params: 354
Trainable params: 354
Non-trainable params: 0
_________________________________________________________________


## CNN(Convolution Neural Network) 학습 및 평가 방법

In [9]:
tf.random.set_seed(777)

Data      = Training_Fold1
Data_Val  = Validation_Fold1
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label

model = CNN_model(Data)

hist = model.fit(Data, Label, epochs=iteration, verbose = 0)
Loss, Accuracy = model.evaluate(Data_Val,  Label_Val, verbose=0)

Accuracy

0.828125

## CNN(Convolution Neural Network) 학습 및 검증

In [10]:
Label        = TrainingFold_Label
Label_Val    = ValidationFold_Label
Accuracy_sum = 0

print('[Reslut of K-fold Cross Validation] \n')

for i in range(Fold):
    tf.random.set_seed(777)
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)
    
    model = CNN_model(Data)
    
# train my model
    hist = model.fit(Data, Label, epochs=iteration, verbose = 0)
    Loss, Accuracy = model.evaluate(Data_Val,  Label_Val, verbose=0)
    Accuracy_sum = Accuracy_sum + Accuracy
    print('Fold {}: {:.2f}%'.format((i+1), Accuracy*100))

print('* Average accuracy : {:.2f}%'.format((Accuracy_sum/Fold)*100))

[Reslut of K-fold Cross Validation] 

Fold 1: 82.81%
Fold 2: 95.31%
Fold 3: 96.88%
Fold 4: 93.75%
Fold 5: 82.81%
* Average accuracy : 90.31%


## 전체 학습 데이터로 NN 학습 및 모델 저장

In [11]:
PrintAccPerEpochs = 200

class AccuracyPerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%PrintAccPerEpochs == 0:
            print("{} Epochs Accuracy : {:.2f}% ".format(epoch, logs["accuracy"]*100))

In [22]:
tf.random.set_seed(777)    

Data  = Training_All
Label = Training_All_Label

model = CNN_model(Training_All)
hist = model.fit(Data, Label, epochs=iteration, verbose = 0, callbacks = [AccuracyPerEpoch()])

print('Final Accuracy      : {:.2f}%'.format(hist.history['accuracy'][-1]*100))

0 Epochs Accuracy : 48.75% 
200 Epochs Accuracy : 61.25% 
400 Epochs Accuracy : 85.31% 
Final Accuracy      : 91.25%


In [13]:
# 학습 과정 확인

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss', c = 'tab:red')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc', c = 'tab:blue')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='lower left')

plt.show()

In [14]:
# 모델 저장
model.save("./MLmodels/Simple_CNN_model.h5")

## NN model 불러와서 진단

In [15]:
# 모델 불러오기
LoadedModel = keras.models.load_model("./MLmodels/Simple_CNN_model.h5")

In [16]:
temp_Test_Data         = np.array(pd.read_csv('./K_FoldData/CNN_Test_Data' , sep=",", header=None))
Test_Label_forANN = np.array(pd.read_csv('./K_FoldData/Test_Label_forANN', sep=",", header=None))

In [20]:
f,t,Test_Data = signal.spectrogram(Test_Data, Fs, nperseg = 78)
print("STFT Result            :", Training_All.shape)

# Data Reshape for CNN
Test_Data = Test_Data.reshape(Test_Data.shape[0], Test_Data.shape[1], Test_Data.shape[2], NoOfSensor)
print("Reshape_for_CNN Result :", Test_Data.shape)

STFT Result            : (320, 40, 40, 1)
Reshape_for_CNN Result : (40, 40, 40, 1)


C:\Users\erich\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 78 is greater than input length  = 1, using nperseg = 1
  .format(nperseg, input_length))


In [21]:
Loss, Accuracy = LoadedModel.evaluate(Test_Data,  Test_Label_forANN, verbose=0)
print('[Performance of CNN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

[Performance of CNN model] 

Accuracy : 50.00%


In [19]:
# 예측값 확인
Predict_y = LoadedModel.predict(Test_Data)
pd.DataFrame(Predict_y)

,0,1
0,0.526471,4.735288e-01
1,0.541835,4.581653e-01
2,0.533428,4.665720e-01
3,0.589855,4.101445e-01
4,0.463598,5.364016e-01
5,0.553360,4.466401e-01
6,0.557973,4.420271e-01
7,0.506076,4.939244e-01
8,1.000000,2.767697e-25
9,0.532246,4.677538e-01
